In [1]:
import spectHR as cs
import pyhrv
from ipywidgets import Tab, Output

In [2]:
cs.handler.show_logs()

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [3]:
DataSet = cs.SpectHRDataset("SUB_008.xdf", use_webdav=True)

In [4]:
DataSet = cs.borderData(DataSet)
#DataSet.ecg = DataSet.ecg.slicetime(500, 570)

In [5]:
DataSet = cs.filterECGData(DataSet, {"filterType": "highpass", "cutoff": .50})

In [6]:
DataSet = cs.calcPeaks(DataSet)

In [7]:
%matplotlib widget
GUI = cs.prepPlot(DataSet, 500, 520)

In [8]:
preProcessing =  Output()
with preProcessing:
    display(GUI)

In [9]:
poincarePlot = Output()

In [10]:
descriptives = Output()

In [11]:
import pyhrv
def helper_welch_psd(RTops = None, mode = 'dev'):
    titlestring = RTops['epoch'].iloc[0]
    res = None
    try:
        res = pyhrv.frequency_domain.welch_psd(RTops['ibi'])
    except:
        pass

    if res is not None:
        pfig = res['fft_plot']
        pfig.set_figwidth(12)
        pfig.set_figheight(5)
        pfig.get_axes()[0].set_title(titlestring)
        pfig.get_axes()[0].set_ylim(0,.1)

psdPlot = Output()

In [12]:
App = Tab(children=[preProcessing, poincarePlot, descriptives, psdPlot])

App.set_title(0, "PreProcessing")
App.set_title(1, "Poincare")
App.set_title(2, "descriptives")
App.set_title(3, "PSD")

def on_tab_change(change):
    if change['name'] == 'selected_index':
        if change['new'] == 1:  # Poincare selected
            with poincarePlot:
                poincarePlot.clear_output()
                display(cs.poincare(DataSet))
        if change['new'] == 2:  # descriptives selected
            with descriptives:
                descriptives.clear_output()
                display(DataSet.RTops.groupby('epoch')['ibi'].agg([len, 'mean', 'std', 'min', 'max', \
                                                                   pyhrv.time_domain.rmssd, pyhrv.time_domain.sdnn, \
                                                                   cs.Tools.Params.sdsd, \
                                                                   cs.Tools.Params.sd1, cs.Tools.Params.sd2, cs.Tools.Params.sd_ratio, \
                                                                   cs.Tools.Params.ellipse_area]))
        if change['new'] == 3:  # psd selected
            with psdPlot:
                psdPlot.clear_output()
                display(DataSet.RTops.groupby('epoch').apply(helper_welch_psd))

                
# Attach observer to detect tab switch
App.observe(on_tab_change, names='selected_index')


In [13]:
display(App)